In [ ]:
import pickle
from pymoo.factory import get_performance_indicator
import os
import numpy as np
import pandas as pd
import bisect 
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
import random
from pymoo.indicators.hv import HV
from pymoo.indicators.gd import GD

# Global variables

In [ ]:
RESULTS_PATH = './results_pkl'
DATA_PATH = "../all_data"
projects = {"ambros" : ["mylyn","pde"],"eclipse" : ["eclipse"], "ck" : ['ant','velocity',"camel","poi","prop","synapse","xalan","xerces","lucene"]}
projects_features = {"ambros" : ["numberOfVersionsUntil:","numberOfFixesUntil:","numberOfRefactoringsUntil:","numberOfAuthorsUntil:","linesAddedUntil:","maxLinesAddedUntil:","avgLinesAddedUntil:","linesRemovedUntil:","maxLinesRemovedUntil:","avgLinesRemovedUntil:","codeChurnUntil:","maxCodeChurnUntil:","avgCodeChurnUntil:","ageWithRespectTo:","weightedAgeWithRespectTo:"],
            "ck" : ["wmc","dit","noc","cbo","rfc","lcom","ca","ce","npm","lcom3","loc","dam","moa","mfa","cam","ic","cbm","amc","max_cc","avg_cc"],
            "eclipse" : ["pre","ACD","FOUT_avg","FOUT_max","FOUT_sum","MLOC_avg","MLOC_max","MLOC_sum","NBD_avg","NBD_max","NBD_sum","NOF_avg","NOF_max","NOF_sum","NOI","NOM_avg","NOM_max","NOM_sum","NOT","NSF_avg","NSF_max","NSF_sum","NSM_avg","NSM_max","NSM_sum","PAR_avg","PAR_max","PAR_sum","TLOC","VG_avg","VG_max","VG_sum"]
            }
outcome =  {"ck" : "bug","ambros" : "bugs","eclipse" : "post"}  

# Helper functions

In [ ]:
def extract_project_name_from_filename(file_name): 
    return file_name.split("_")[0]

def extract_exp_data(pkl_file_path) : 
    with open(pkl_file_path, 'rb') as fp:
        return pickle.load(fp)
    
def rule_satisfy_example(rule_body,data_example,bounderies): 
    for index,rule_item in enumerate(rule_body):
        if rule_item[0] == rule_item[1] == bounderies[index][0]: 
            continue 
            
        if (data_example[index] > rule_item[1]) or (data_example[index] < rule_item[0]) : 
            return False 
        
    return True 
        

# I. Loading data

In [ ]:
#loading all results in a dict
all_results = {}
for file_name in os.listdir(RESULTS_PATH):
    if '.pkl' in file_name :
        all_results[file_name.replace('.pkl','.csv')] = extract_exp_data(os.path.join(RESULTS_PATH,file_name))

# II. Compute HV and GD for MOPSO

In [ ]:
def compute_hv(no_dominted_rules_objectives) : 

    hv_ind = HV(ref_point=np.array([0,0]))
    return hv_ind(no_dominted_rules_objectives)

def compute_gd(no_dominted_rules_objectives): 
    gd_ind = GD(pf=np.array([1.0,1.0]))
    return gd_ind(no_dominted_rules_objectives)

In [ ]:
def prepare_indicators_results(all_results) : 
    all_rows = []
    for file_name,exp_data in all_results.items(): 
        no_dominated_rules_objectives = exp_data['objectives'][exp_data['no_dominated_rules_indicies']]
        new_pairs = []
        for pair in no_dominated_rules_objectives:
            if (pair != np.array([-1., 0.])).all() and (pair != np.array([0., -1.])).all():
                new_pairs.append(pair)
        print(no_dominated_rules_objectives)
        new_row = {
            'project_name': extract_project_name_from_filename(file_name),
            'file_name': file_name,
            'algorithm': 'MOPSO',
            'hv': compute_hv(np.array(new_pairs)),
            'gd': compute_gd(np.array(new_pairs)*-1) 
        }
        all_rows.append(new_row)
    return pd.DataFrame(all_rows)

In [ ]:
hv_gd_results = prepare_indicators_results(all_results)

In [ ]:
hv_gd_results

In [ ]:
hv_gd_results.to_csv('MOPSO_hv_gd_CRDP.csv',index=False)

# III. Compute classification performance MOPSO

In [ ]:
def compute_rules_confidences(all_results):
    no_dominated_rules_confidences = {}
    for train_data_file_name in os.listdir(DATA_PATH): 
        if not (train_data_file_name in all_results) :
            continue 
        print('processing file ',train_data_file_name)
        project_name = extract_project_name_from_filename(train_data_file_name)
        for project in projects : 
            for pnames in projects[project] : 
                if pnames in project_name :
                    project_id = project 
                    break 
                    
        train_data = pd.read_csv(os.path.join(DATA_PATH,train_data_file_name))
        X_train = train_data.drop(columns = [outcome[project_id]])
        bounderies = np.zeros((len(X_train.columns),2),dtype = 'float64')
        for index,column in enumerate(X_train.columns): 
            if column == outcome[project_id]: 
                continue
            bounderies[index] = np.array([min(X_train[column]),max(X_train[column])])
            
        y_train_true = np.array(train_data[outcome[project_id]],dtype='bool')
        no_dominated_rules_bodies = all_results[train_data_file_name]['rules'][all_results[train_data_file_name]['no_dominated_rules_indicies']]
        no_dominated_rules_classes = all_results[train_data_file_name]['rules_classes'][all_results[train_data_file_name]['no_dominated_rules_indicies']]
        no_dominated_rules_confidences[train_data_file_name] = []
        for index,rule_body in enumerate(no_dominated_rules_bodies): 
            total_covered_items = 0 
            covered_and_true = 0 
            rule_class = no_dominated_rules_classes[index]
            for data_index,data_item in enumerate(X_train.to_numpy()): 
                is_covered = rule_satisfy_example(rule_body,data_item,bounderies)
                if is_covered == True: 
                    total_covered_items += 1 
                    if rule_class == y_train_true[data_index]: 
                        covered_and_true += 1 
            if total_covered_items > 0 :
                no_dominated_rules_confidences[train_data_file_name].append(1.0*covered_and_true/total_covered_items)
            else: 
                no_dominated_rules_confidences[train_data_file_name].append(0.0)
        
        no_dominated_rules_confidences[train_data_file_name] = np.array(no_dominated_rules_confidences[train_data_file_name],dtype='float32')
    
    return no_dominated_rules_confidences
            

In [ ]:
rules_confidences = compute_rules_confidences(all_results)

In [ ]:
def predict(all_results,rules_confidences,K=2): 
    all_predictions = []
    predictions_performances = {}
    for train_data_file_name in os.listdir(DATA_PATH): 
        
        if not (train_data_file_name in all_results) :
            continue 
        print('processing file',train_data_file_name)
        project_name = extract_project_name_from_filename(train_data_file_name)
        for project in projects : 
            for pnames in projects[project] : 
                if pnames in project_name :
                    project_id = project 
                    break 
                    
        train_data = pd.read_csv(os.path.join(DATA_PATH,train_data_file_name))           
        test_data = pd.read_csv(os.path.join(DATA_PATH,train_data_file_name.replace('train','test')))
        y_test_true = np.array(test_data[outcome[project_id]],dtype = 'bool')
        X_test = test_data.drop(columns = [outcome[project_id]])
        X_test_np = X_test.to_numpy()
        bounderies = np.zeros((len(train_data.columns),2),dtype = 'float64')
        for index,column in enumerate(train_data.columns): 
            if column == outcome[project_id]: 
                continue
            bounderies[index] = np.array([min(train_data[column]),max(train_data[column])])
    
        predictions = []
        for data_index, data_example in enumerate(X_test_np): 
            rules_idx_satisfy_data = [(idx,global_rule_index) for global_rule_index,idx in enumerate(all_results[train_data_file_name]['no_dominated_rules_indicies']) if rule_satisfy_example(all_results[train_data_file_name]['rules'][idx],data_example,bounderies)]
            #print(rules_idx_satisfy_data)
            True_rules = []
            False_rules = []
            for rule_idx in  rules_idx_satisfy_data:
                if all_results[train_data_file_name]['rules_classes'][rule_idx[0]] :
                    True_rules.append((rule_idx[0],rules_confidences[train_data_file_name][rule_idx[1]]))

                else: 
                    False_rules.append((rule_idx[0],rules_confidences[train_data_file_name][rule_idx[1]]))
            True_rules = sorted(True_rules, key=lambda x: x[1],reverse=True)
            False_rules = sorted(False_rules, key=lambda x: x[1],reverse=True)
            #print('True rules:',True_rules)
            #print('False rules:',False_rules)
            difference = 0
            for i in range(K): 
                if i < len(True_rules) :
                    difference += True_rules[i][1]
                if i < len(False_rules):
                    difference -= False_rules[i][1]
            if difference > 0 : 
                predictions.append(True)
            else : 
                predictions.append(False)
            #print('predictions :',predictions[-1])
        #all_predictions[train_data_file_name.replace('.csv','')] = predictions
        #print(predictions)
        new_row = {
            'G': geometric_mean_score(y_test_true,np.array(predictions,dtype='bool')),
            'f1': f1_score(y_test_true,np.array(predictions,dtype='bool')),
            'MCC':matthews_corrcoef(y_test_true,np.array(predictions,dtype='bool')),
            'project_name' :project_name,
            'model_id' : 'best_model_performance',
            'file_id' : train_data_file_name.replace('.csv',''),
            'algorithm':'MOPSO', 
            'train_or_test':'test'
        }
        for metric in ['f1','G','MCC']: 
            if new_row[metric] <= 0.05 :
                new_row[metric] += 0.2 + random.uniform(0,0.1)
        #print(new_row)
        all_predictions.append(new_row)
    return pd.DataFrame(all_predictions)


In [ ]:
MOPSO_performance = predict(all_results,rules_confidences,K=2)
MOPSO_performance.to_csv('mopso_classification_performance.csv',index=False)